# Table of Contents

This notebook generates data used to contruct `binding peak` tables and `venn diagrams` for ChipDB. But before you run the notebook, make sure of the following: 

1. Concatnate files that contain multiple conditions for the same TF by using the following command on your terminal: `cat /path/prefix* >> new_file`.
2. Make sure to rename TF in the gff file to match the TF convention and not gene convension.

In [1]:
#basics:
import numpy as np     
import pandas as pd
import urllib                      
import gzip
import os
from os import path
from collections import defaultdict

# Binding Peak Tables

## Generate a data objects

### Prerequisit
Ensure the following paths reflect your local **input** and **output** directories

In [2]:
data_dir = './'

In [3]:
in_data = data_dir+ 'input_data/'
in_gff = in_data+ 'gff_data/'
out_data_venn = data_dir+ 'output_data/'+'venn_data/'
out_data_table = data_dir+ 'output_data/'+'table_data/'

In [4]:
database = defaultdict(list)

In [22]:
TF_list

['YbeF',
 'YciT',
 'YdcI',
 'YbcM',
 'YddM',
 'YdiP',
 'YbhD',
 'YieP',
 'YcfQ',
 'YcaN',
 'YbaQ',
 'YdhB',
 'YeaM',
 'YjdC',
 'YdcN',
 'YdcI',
 'YheO',
 'YdcR',
 'YbaO',
 'YcjW',
 'YahB',
 'YbdO',
 'YdcI']

In [5]:
filelist = [file for file in os.listdir(in_gff)]
filelist

['Ybef_M9_curated.gff',
 'Ycit_M9_curated.gff',
 'YdcI_ph5_curated.gff',
 'Ybcm_M9_curated.gff',
 'Yddm_M9_curated.gff',
 'Ydip_M9_curated.gff',
 'Ybhd_M9_curated.gff',
 'Yiep_M9_curated.gff',
 'Ycfq_M9_curated.gff',
 'Ycan_M9_curated.gff',
 'Ybaq_M9_curated.gff',
 'Ydhb_M9_curated.gff',
 'Yeam_M9_curated.gff',
 'Yjdc_M9_curated.gff',
 'Ydcn_M9_curated.gff',
 'YdcI_ph8_curated.gff',
 'Yheo_M9_curated.gff',
 'Ydcr_M9_curated.gff',
 'Ybao_M9_curated.gff',
 'Ycjw_M9_curated.gff',
 'Yahb_M9_curated.gff',
 'Ybdo_M9_curated.gff',
 'YdcI_ph7_curated.gff']

In [6]:
filelist = [file for file in os.listdir(in_gff)]

for i in range(0,len(filelist)):
    df = pd.read_csv(path.join(in_gff,filelist[i]),index_col=0, 
                                 delimiter='\t', header=None, 
                                 names = ['ref','condition', 'condition_name', 
                                          "binding_peak_start",'binding_peak_end', 
                                          'binding_peak_strength', 'direction', '.','ID'])

    df = df.set_index(pd.Series(range(1,len(df)+1)))
    database[i] = df

In [24]:
database[13].head()

,condition,condition_name,binding_peak_start,binding_peak_end,binding_peak_strength,direction,.,ID
1,MACE,_filtered_0.95,201,249,2.91,+,.,SRB
2,MACE,_filtered_0.95,681,726,1.44,+,.,SFB
3,MACE,_filtered_0.95,945,992,1.22,+,.,SFB
4,MACE,_filtered_0.95,996,1042,1.26,+,.,SRB
5,MACE,_filtered_0.95,2030,2064,1.03,+,.,SRB


For the next block of code to work, make sure the files are named in the correct format: 
    `"<TF>_<CONDITION>_cutared.gff"`. The code will ensure TFs, genes, and conditions are in the correct conventions.

In [8]:
TF_list =[]
TF_condition = []
TF_gene_name = []
for i in range (0,len(filelist)): 
    split_file_name = filelist[i].split('_')
    
    first_upper_case = split_file_name[0].replace(split_file_name[0][0], split_file_name[0][0].upper())
    last_upper_case = first_upper_case.replace(first_upper_case[-1], first_upper_case[-1].upper())
    TF_list.append(last_upper_case)
    
    TF_condition.append(split_file_name[1])
    
    TF_gene_name.append(last_upper_case.lower())

## Load gene info + Biocyc TRN 

In [9]:
DF_EcoCyc_TF = pd.read_csv(path.join(data_dir,'input_data','trn_EcoCyc.csv'),index_col=0)
DF_gene_info = pd.read_csv(path.join(data_dir,'input_data','gene_info.csv'),index_col=0)
precise2_TRN = pd.read_csv(path.join(data_dir,'input_data','trn_precise2.csv'),index_col=0)

In [10]:
# Define start codon position (to account for +/- strands)
DF_gene_info['start_codon_pos'] = [row.start if row.strand == '+' else row.stop for idx,row in DF_gene_info.iterrows()]
DF_gene_info.head()

,start,stop,strand,gene_name,length,operon,cog,start_codon_pos
b0001,189,255,+,thrL,66,thrLABC,No COG Annotation,189
b0002,336,2799,+,thrA,2463,thrLABC,No COG Annotation,336
b0003,2800,3733,+,thrB,933,thrLABC,Amino acid transport and metabolism,2800
b0004,3733,5020,+,thrC,1287,thrLABC,Amino acid transport and metabolism,3733
b0005,5233,5530,+,yaaX,297,yaaX,Function unknown,5233


## Generate Binding peaks 

This function assigns bindings peaks for each TF and fins the corresponsing gene targets using the `gene_info.csv` table. 

In [11]:
def annotate_peaks(TF_name, TF_condition, peak_df,margin,gene_info):
    
    res_df = peak_df.copy()
#     TF = peak_df.condition_name[1][:4]
    for i,row in res_df.iterrows():
        pos = row['binding_peak_start']
        # Identify genes within MARGIN nt of binding peak
        close_genes = gene_info[(gene_info.start_codon_pos > pos-margin) 
                                & (gene_info.start_codon_pos < pos+margin)]
        for strand,group in close_genes.groupby('strand'):
            #Remove genes that are completely transcribed before binding peak
            if strand == '+':
                group = group[group.stop > pos]
            else:
                group = group[group.start < pos]

            operon = group.operon.unique()
            # Ensure that we're only identifying one operon on either side of binding peak
            if len(operon) > 1:
                print (operon)

            # Get all genes in operon
            bnums = gene_info[gene_info.operon.isin(operon)].index
            
            ## Add gene information to dataframe
            if strand == '+':
                res_df.loc[i,'TU_p'] = ','.join(operon)
                res_df.loc[i,'genes_p'] = ','.join(bnums)
            else:
                res_df.loc[i,'TU_m'] = ','.join(operon)
                res_df.loc[i,'genes_m'] = ','.join(bnums)
    res_df['index'] = [TF_name +'-' + str(i) for i in range(1,peak_df.shape[0]+1)]
    res_df['condition'] = [ TF_name.lower() + " + " + TF_condition for i in peak_df.condition_name]
#     [peak_df.condition_name[2][:4]+' + '+peak_df.condition_name[1][5:8]]*peak_df.shape[0]
    cols = ['index','condition','binding_peak_start','binding_peak_end',
            'binding_peak_strength','TU_p','genes_p','TU_m','genes_m']
    return res_df.reindex(columns = cols)

## Validate Binding Peaks 

this function validates the accuracy of every gene target from the already indetified gene list for every binding site, identified from the previous function  

In [12]:
def validate_peak_info(df,gene_info):
    locusTag = defaultdict(list)
    geneName = defaultdict(list)
    for i,row in df.iterrows():
        BP = row['binding_peak_start']
        idx_name = i
        genes = [row['genes_p'] , row['genes_m']]
        if ((genes[0] == '') & (genes[1] == '')):
            locusTag[idx_name].append('')
            geneName[idx_name].append('')
        for gene in genes:
            if gene == '':
                continue 
            gene_list = gene.split(',')
            for g in gene_list: 
                name = DF_gene_info.loc[g].gene_name 
                strand = DF_gene_info.loc[g].strand 
                start = DF_gene_info.loc[g].start
                stop = DF_gene_info.loc[g].stop
                if ((start > BP) & (stop > BP) & (strand == '+')) | ((start < BP) & (stop < BP) & (strand == '-')):
                        locusTag[idx_name].append(g)
                        geneName[idx_name].append(name)
                elif ((start < BP) & (stop > BP)) | ((start > BP) & (stop < BP)):
                    locusTag[idx_name].append(g)
                    geneName[idx_name].append(name)
                    
    for k, v in locusTag.items():
        if ((len(v) == 1) & (v[0] == '')): 
            locusTag[k] = ''
            continue
        genes = ','.join(locusTag[k])
        locusTag[k] = genes

    for k, v in geneName.items():
        if ((len(v) == 1) & (v[0] == '')): 
            geneName[k] = ''
            continue
        genes = ','.join(geneName[k])
        geneName[k] = genes

    df_complete = df.loc[:,['index','condition','binding_peak_start',
                            'binding_peak_end','binding_peak_strength']]
    print(locusTag.values())
    df_complete['target_locus'] = locusTag.values()
    df_complete['target_genes'] = geneName.values()
    return df_complete

# Venn Diagrams

Make sure to update the list of `TF_names` as Ye adds more gff files into the dropbox

In [13]:
TRN_data = defaultdict(list)
TF_name = TF_list

for i in TF_name: 
    TRN_data[i] = [x for x in  list(precise2_TRN.gene_name[precise2_TRN.index == i])  if  str(x) != 'nan']

In [14]:
def Venn_data_gen(Peak_DF): 
    TF_name = Peak_DF['index'][1][:4]
    gene_list = [i for i in Peak_DF.target_genes if i != '']
    chip_data= ','.join(list(gene_list)).split(',')
    reg_data = TRN_data[TF_name]
    all_genes = [i  for i in chip_data if i not in reg_data] + reg_data
    
    
    TF = TF_name
    reg_genes=reg_data
    reg_only = []
    chip_genes=chip_data
    chip_only = []
    shared_genes=[]
    for i in all_genes: 
        if (i in reg_data) & (i not in chip_data):
            reg_only.append(i)
        elif (i in chip_data) & (i not in reg_data):
            chip_only.append(i)
        elif (i in chip_data) & (i in reg_data):
            shared_genes.append(i)
            
    values = [TF,
          len(reg_genes),
          len(reg_only),
          len(chip_genes),
          len(chip_only),
          len(shared_genes),
          len(all_genes)]
    
    index_name = ['TF',
              'reg_genes',
              'reg_only',
              'chip_genes',
              'chip_only',
              'shared_genes',
              'all_genes']
    
    genes = ['; '.join(precise2_TRN.source[precise2_TRN.index == TF_name].unique()),
         reg_genes,
         reg_only,
         chip_genes,
         chip_only,
         shared_genes,
         all_genes]
    
    same1 = defaultdict(list)
    for i in range(0,len(index_name)):
        same1[index_name[i]].append(values[i])

    finall = pd.DataFrame.from_dict(same1, orient='index', columns = ['value'])
    finall['list'] = genes
    for i, row in finall.iterrows(): 
        if row.value == 0:
            finall.list[i] = ''
    return finall

In [15]:
'; '.join(precise2_TRN.source[precise2_TRN.index == 'YdcI'].unique())

'Gao2021; Ecocyc'

In [23]:
for i in TF_list:
    print(i,len(TRN_data[i]))

YbeF 3
YciT 49
YdcI 21
YbcM 12
YddM 19
YdiP 5
YbhD 1
YieP 6
YcfQ 3
YcaN 6
YbaQ 38
YdhB 28
YeaM 4
YjdC 0
YdcN 61
YdcI 21
YheO 8
YdcR 19
YbaO 3
YcjW 73
YahB 3
YbdO 4
YdcI 21


In [17]:
i = 16
print(TF_list[i])
peak_annot_DF = annotate_peaks(TF_list[i], TF_condition[i],database[i],500,DF_gene_info).fillna('')
final_annot_DF = validate_peak_info(peak_annot_DF,DF_gene_info)
venn_files = Venn_data_gen(final_annot_DF)

YheO
dict_values(['b0151,b0152,b0153', 'b0628', 'b0628', 'b1381,b1382,b1383,b1380', 'b1429,b1430,b1428', 'b2296,b2297,b2295', 'b2296,b2297,b2295', 'b3710'])


In [18]:
peak_annot_DF

,index,condition,binding_peak_start,binding_peak_end,binding_peak_strength,TU_p,genes_p,TU_m,genes_m
1,YheO-1,yheo + M9,169871,169917,3.30,fhuACDB,"b0150,b0151,b0152,b0153",,
2,YheO-2,yheo + M9,660252,660295,19.75,,,lipA,b0628
3,YheO-3,yheo + M9,660263,660304,19.35,,,lipA,b0628
4,YheO-4,yheo + M9,1443033,1443078,27.17,ydbH-ynbE-ydbL,"b1381,b1382,b1383",ldhA,b1380
5,YheO-5,yheo + M9,1500527,1500575,5.28,tehAB,"b1429,b1430",ydcK,b1428
6,YheO-6,yheo + M9,2413131,2413173,101.47,ackA-pta,"b2296,b2297",yfbV,b2295
7,YheO-7,yheo + M9,2413141,2413183,102.83,ackA-pta,"b2296,b2297",yfbV,b2295
8,YheO-8,yheo + M9,3891510,3891557,13.10,mdtL,b3710,,


In [19]:
final_annot_DF

,index,condition,binding_peak_start,binding_peak_end,binding_peak_strength,target_locus,target_genes
1,YheO-1,yheo + M9,169871,169917,3.30,"b0151,b0152,b0153","fhuC,fhuD,fhuB"
2,YheO-2,yheo + M9,660252,660295,19.75,b0628,lipA
3,YheO-3,yheo + M9,660263,660304,19.35,b0628,lipA
4,YheO-4,yheo + M9,1443033,1443078,27.17,"b1381,b1382,b1383,b1380","ydbH,ynbE,ydbL,ldhA"
5,YheO-5,yheo + M9,1500527,1500575,5.28,"b1429,b1430,b1428","tehA,tehB,ydcK"
6,YheO-6,yheo + M9,2413131,2413173,101.47,"b2296,b2297,b2295","ackA,pta,yfbV"
7,YheO-7,yheo + M9,2413141,2413183,102.83,"b2296,b2297,b2295","ackA,pta,yfbV"
8,YheO-8,yheo + M9,3891510,3891557,13.10,b3710,mdtL


In [20]:
venn_files

,value,list
TF,YheO,Gao2021
reg_genes,8,"[ackA, yfbV, tehA, ydcK, ydbH, ldhA, lipA, pta]"
reg_only,0,
chip_genes,19,"[fhuC, fhuD, fhuB, lipA, lipA, ydbH, ynbE, ydb..."
chip_only,7,"[fhuC, fhuD, fhuB, ynbE, ydbL, tehB, mdtL]"
shared_genes,8,"[ackA, yfbV, tehA, ydcK, ydbH, ldhA, lipA, pta]"
all_genes,15,"[fhuC, fhuD, fhuB, ynbE, ydbL, tehB, mdtL, ack..."


## Generate the data

Run this block if you are ready to generate your data for ChIPdb.

In [21]:
for i in range (0, len(database)): 
    peak_annot_DF = annotate_peaks(TF_list[i], TF_condition[i],database[i],500,DF_gene_info).fillna('')
    final_annot_DF = validate_peak_info(peak_annot_DF,DF_gene_info)
    venn_files = Venn_data_gen(final_annot_DF)
    final_annot_DF.to_json(out_data_table+TF_list[i]+'_binding_table.json',orient='records')
    venn_files.to_json(path.join(out_data_venn, TF_list[i]+'_venn.json'),orient='records')

dict_values(['b0826', 'b2580,b2579', ''])
['rpiA' 'yqfE']
dict_values(['b0258', 'b0258', 'b0419', 'b0436', '', 'b0808', '', 'b0821', 'b0822', 'b0825,b0823,b0824', 'b0854,b0855,b0856,b0857', 'b0854,b0855,b0856,b0857', 'b0894,b0895,b0896', 'b0907,b0908', '', 'b1042,b1043', 'b1078,b1079,b1080,b1081', '', 'b4672,b1285', '', 'b1521', 'b1853,b1852', '', 'b2063,b2058,b2059,b2060,b2061,b2062', 'b2289', '', 'b2340', 'b2361,b2362,b2363', 'b2369,b2370,b2367,b2368', '', 'b2448,b2449', 'b2450', '', 'b2504,b2503', 'b2702,b2703,b2704,b2705,b2706,b2707,b2708,b2701', 'b2916,b2914,b2915', 'b3136,b3137,b3138,b3139,b3140,b3141', 'b3158,b3159,b3156,b3157', 'b3188', 'b3324,b3325,b3326,b3327,b3328,b3329,b3330,b3331,b3332,b3333,b3334,b3335', 'b3376,b3377,b3378', 'b3397', '', 'b4476,b3437', 'b3589', 'b3749,b3750,b3751,b3752,b3753', 'b4109,b4110,b4108', 'b4190,b4189', 'b4193,b4194,b4195,b4196,b4197,b4198,b4192', 'b4295', '', 'b4342', 'b4567,b4367', 'b4381,b4382,b4383,b4384,b4380'])
['holE' 'yobB-exoX']
dict_val

['holE' 'yobB-exoX']
dict_values(['b0019,b0020,b0018,b4412', 'b0076,b0071,b0072,b0073,b0074,b0075', 'b0281', 'b0382,b0381', 'b0720', 'b0770', 'b1418,b4493_2,b4493_1', 'b1634', 'b1823,b1824', 'b1842,b1843,b1844,b1839,b1840,b1841', 'b4496_1,b4496_2,b1962,b1963', 'b2210', '', 'b3267', 'b3603,b3604,b3605', 'b3646', 'b3710', 'b3800', 'b4077', 'b4077', 'b4145'])
dict_values(['b0151,b0152,b0153', 'b0628', 'b0628', 'b1381,b1382,b1383,b1380', 'b1429,b1430,b1428', 'b2296,b2297,b2295', 'b2296,b2297,b2295', 'b3710'])
['ylaC' 'maa']
['hicAB' 'ydcR']
dict_values(['b0001,b0002,b0003,b0004', 'b0458,b0459', 'b1375', 'b1438,b1439', 'b1461', 'b1544,b4600_2', 'b1678', 'b1770', 'b1823,b1824', 'b1987', 'b2371,b2372,b2373', '', 'b2975', 'b3243,b3240,b3241,b3242', 'b3475', '', 'b3645', 'b3656', '', 'b4296'])
['decR' 'mdlAB']
dict_values(['b0447,b0448,b0449', 'b4470,b3110'])
['allA' 'allR']
['insH-8' 'yejO']
dict_values(['', 'b0066,b0067', 'b0114,b0115,b0116', 'b0144,b0145', 'b0151,b0152,b0153', 'b0231,b0232,b